In [4]:
import streamlit as st
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_absolute_percentage_error
import shap
import matplotlib.pyplot as plt


In [5]:
# Fungsi untuk membuat dataset dengan look_back
def create_dataset(X, Y, look_back=1):
    Xs, Ys = [], []
    for i in range(len(X) - look_back - 1):
        Xs.append(X[i:(i + look_back)])
        Ys.append(Y[i + look_back])
    return np.array(Xs), np.array(Ys)

# Model LSTM
def build_lstm_model(input_shape, units, dropout):
    model = Sequential()
    model.add(LSTM(units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(units))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam')
    return model

In [6]:

# Load data
df = pd.read_excel("DATA FIKS.xlsx")
dataY = df['Price'].values.reshape(-1, 1)
dataX = df.drop(['Date', 'Price'], axis=1).values  # drop columns you don't want
feature_names = df.drop(['Date', 'Price'], axis=1).columns.tolist()

# Standarisasi data
scalerX, scalerY = StandardScaler(), StandardScaler()
dataX = scalerX.fit_transform(dataX)
dataY = scalerY.fit_transform(dataY)


In [12]:
# Buat model
look_back = 1
train_size = int(len(dataX) * 0.8)
trainX, testX = dataX[:train_size], dataX[train_size:]
trainY, testY = dataY[:train_size], dataY[train_size:]

trainX, trainY = create_dataset(trainX, trainY, look_back)
testX, testY = create_dataset(testX, testY, look_back)

# Bentuk ulang input menjadi 3D
trainX = trainX.reshape(trainX.shape[0], look_back, trainX.shape[2])
testX = testX.reshape(testX.shape[0], look_back, testX.shape[2])

In [13]:

# Dashboard menggunakan Streamlit
st.title("Prediksi Harga Bitcoin")

# Pilihan variabel input dari pengguna
selected_features = st.multiselect('Pilih variabel input', feature_names)

# Jika tidak ada variabel yang dipilih, berikan pesan
if len(selected_features) == 0:
    st.write("Silakan pilih setidaknya satu variabel untuk memulai prediksi.")

else:
    # Filter data sesuai dengan fitur yang dipilih
    selected_indices = [feature_names.index(f) for f in selected_features]

    # Update dataX sesuai variabel yang dipilih
    dataX_selected = df[selected_features].values
    dataX_selected_scaled = scalerX.fit_transform(dataX_selected)

    # Bagi data ke dalam train/test
    trainX_selected, testX_selected = dataX_selected_scaled[:train_size], dataX_selected_scaled[train_size:]
    trainX_selected, trainY_selected = create_dataset(trainX_selected, dataY[:train_size], look_back)
    testX_selected, testY_selected = create_dataset(testX_selected, dataY[train_size:], look_back)

    trainX_selected = trainX_selected.reshape(trainX_selected.shape[0], look_back, trainX_selected.shape[2])
    testX_selected = testX_selected.reshape(testX_selected.shape[0], look_back, testX_selected.shape[2])

    # Buat model
    model = build_lstm_model((look_back, trainX_selected.shape[2]), units=100, dropout=0.3)
    model.fit(trainX_selected, trainY_selected, epochs=50, batch_size=16, verbose=0, shuffle=False)

    # Prediksi menggunakan data testing
    testPredict_scaled = model.predict(testX_selected)
    testPredict = scalerY.inverse_transform(testPredict_scaled)
    testY_actual = scalerY.inverse_transform(testY_selected)

    # Hitung MAPE
    test_mape = mean_absolute_percentage_error(testY_actual, testPredict)
    st.write(f"Loss (MAPE) pada data testing: {test_mape:.2f}")

    # Plot hasil prediksi vs aktual
    st.subheader("Grafik Prediksi vs Data Aktual")
    fig, ax = plt.subplots()
    ax.plot(testY_actual, label="Data Aktual")
    ax.plot(testPredict, label="Prediksi")
    ax.legend()
    st.pyplot(fig)

    # SHAP interpretation berdasarkan data testing
    st.subheader("Grafik SHAP untuk Interpretasi Model pada Data Testing")
    explainer = shap.KernelExplainer(model.predict, testX_selected)
    shap_values = explainer.shap_values(testX_selected)

    fig, ax = plt.subplots()
    shap.summary_plot(shap_values, feature_names=selected_features, plot_type="bar", show=False)
    st.pyplot(fig)

# Prediksi 1 periode ke depan berdasarkan harga dan volume perdagangan
st.subheader("Prediksi 1 Hari ke Depan")

# Input pengguna untuk prediksi 1 hari ke depan
harga_btc_t1 = st.number_input('Harga Bitcoin t-1', value=0.0, format="%.2f")
volume_btc = st.number_input('Volume Perdagangan Bitcoin', value=0.0, format="%.2f")

# Bentuk array input untuk prediksi 1 hari ke depan
next_input = np.array([harga_btc_t1, volume_btc]).reshape(1, -1)
print(next_input)
next_input_scaled = scalerX.transform(next_input).reshape(1, look_back, next_input.shape[1])

# Prediksi 1 hari ke depan
next_prediction_scaled = model.predict(next_input_scaled)
next_prediction = scalerY.inverse_transform(next_prediction_scaled)

st.write(f"Prediksi Harga Bitcoin 1 Hari ke Depan: {next_prediction[0][0]:.2f}")

[[0. 0.]]


ValueError: X has 2 features, but StandardScaler is expecting 5 features as input.